## BIM to BuildingSceneLayer for SC Station Structures and Depot Buildings

#### This Notebook imports revit files from BIM models and convert them to BuildingSceneLayer in prepartion for smart map
#### Please implement the following steps in preparing building layers to be published to Portal for ArcGIS.
1. Define Projection to PRS92
2. BIM File to Geodatabase
3. Make Building Layer
4. Enable Archiving, GlobalIDs, and Editor Tracking
5. Add Fields for Structural discipline layers
6. Apply symbology to Status
7. Set the initial status = 'To be Constructed'
8. Add station names and CP
9.Assign Domains


### SC Station Structures

In [4]:
import time, os, datetime, sys, logging, logging.handlers, shutil, traceback, re
from pathlib import Path
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984 Web Mercator (auxiliary sphere)")
arcpy.env.geographicTransformations = "PRS_1992_To_WGS_1984_1"

home = Path.home()
directory = r'C:/Users\eiji.LAPTOP-KSD9P6CP/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexsc'
workspace = os.path.join(home, directory)
#workspace = r'C:/Users/oc3512/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb'


In [ ]:
# 1. Define Projection
## Manually check projection of all Revit models of structural disciplines
## Make sure that all the models have projection defined (i.e., at the right location)

In [2]:

BuildingLayerName = "SC_Station_Structure"
out_gdb_path = os.path.join(workspace,"NSCR-Ex.sde")
out_dataset_name = "SC_Station_Structure_BIM"

In [6]:
# 2. BIM File to Geodatabase
## This saves all Revit models to enterprise geodatabase

RevitWD = os.path.join(home, "Dropbox/01-Railway/02-NSCR-Ex/02-SC/01-Basedata/07-Station_Structure/01-BIM/")

Revit1 = os.path.join(RevitWD,"S-01/NSCR-GCR-N01-APLSTN-MOD-ST-000001.rvt")
Revit2 = os.path.join(RevitWD, "S-01/NSCR-GCR-N01-CLPSTN-MOD-ST-000001.rvt")
Revit3 = os.path.join(RevitWD, "S-02/NSCR-GCR-N02-CSFSTN-MOD-ST-000001.rvt")
Revit4 = os.path.join(RevitWD, "S-03/NSCR-GCR-N03-ANGSTN-MOD-ST-000001.rvt")
Revit5 = os.path.join(RevitWD, "S-03/NSCR-GCR-N03-CRKSTN-MOD-ST-000001.rvt")
Revit6 = os.path.join(RevitWD, "S-04/NSCR-GCR-N04-CIASTN-MOD-ST-000001.rvt")

inputRevit = [Revit1,Revit2,Revit3,Revit4,Revit5,Revit6]
# Choose enterprise geodatabase for Output Geodatabase
spatial_reference = "PRS_1992_Philippines_Zone_III"

arcpy.BIMFileToGeodatabase_conversion(inputRevit, out_gdb_path, out_dataset_name, spatial_reference)


<Result 'C:/Users/eiji.LAPTOP-KSD9P6CP/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2\\10.10.5.5.sde\\NSCR_Ex.NSCREXUSER.N2_Station_Structure_BIM'>

In [ ]:
# 3. Make Building Layer
## Convert files created above to building layer
# Make a building layer from a Dataset
# The code below failed. why? but succeeded when it was manually executed in Geoprocessing.
out_dataset = os.path.join(out_gdb_path,"NSCR_Ex.NSCREXUSER.SC_Station_Structure_BIM")
arcpy.MakeBuildingLayer_management(out_dataset,BuildingLayerName)

In [10]:
# 4. Enable Archiving, GlobalIDs, and Editor Tracking by
#    right-clicking enterprise geodatabase created by 'BIM File to Geodatabase' tool

In [ ]:
#------------------------------------------------------------------#
# The following command is intended for CIA station only.
# The reason is that some structural components belong to the architectural. This is wrong.
# Nonetheless, the BIM Team cannot fix this any more. So the only option is to keep, treat, and 
# monitor as the architectural.
# In reality, each Architectural discipline feature layer has information on all the N2 stations to be monitored.
# so the code below is eventually applied to all the station structures but actual use is limited to the
# CIA station for monitoring.

in_features = ["SC_Station_Structure\\Structural\\StructuralColumns",
          "SC_Station_Structure\\Structural\\StructuralFoundation",
          "SC_Station_Structure\\Structural\\StructuralFraming",
          "SC_Station_Structure\\Architectural\\Columns",
          "SC_Station_Structure\\Architectural\\Floors",
          "SC_Station_Structure\\Architectural\\Walls"]

addFields = ['Station', 'Types', 'SubType', 'Status', 'start_date',
             'end_date', 'StructureType', 'StructureLevel', 'P6ID', 'Comment',
            'GridX', 'GridY', 'CP', 'ID']
#--------------------------------------------------------------------------------------#

In [16]:
# 5. Add Fields for Structural discipline layers
## StructuralColumns, StructuralFraming, and StructuralFoundation
for feature in in_features:
    for field in addFields:
        if field in ('start_date', 'end_date'):
            arcpy.AddField_management(feature, field, "DATE", "", "", "", field, "NULLABLE", "")
        elif field in ('P6ID', 'Comment', 'GridX', 'GridY', 'CP', 'ID'):
            arcpy.AddField_management(feature, field, "TEXT", "", "", "", field, "NULLABLE", "")
        else:
            arcpy.AddField_management(feature, field, "SHORT", "", "", "", field, "NULLABLE", "")

In [17]:
# 6. Set the initial status = 'To be Constructed'
for feature in in_features:
    with arcpy.da.UpdateCursor(feature, ['Status']) as cursor:
        for row in cursor:
            row[0] = 1
            cursor.updateRow(row)

In [18]:
# Enter CP
for feature in in_features:
    with arcpy.da.UpdateCursor(feature,['DocName','CP']) as cursor:
        for row in cursor:
            if row[0]:
                try:
                    reg = re.search(searchStns,row[0]).group()
                except AttributeError:
                    reg = re.search(searchStns,row[0])
                
                # Station name
                if reg == "APLSTN":
                    row[1] = "N-01"
                elif reg == "CLPSTN":
                    row[1] = "N-01"
                elif reg == "CSFSTN":
                    row[1] = "N-02"
                elif reg == "ANGSTN":
                    row[1] = "N-03"
                elif reg == "CRKSTN":
                    row[1] = "N-03"
                elif reg == "CIASTN":
                    row[1] = "N-04"
            else:
                row[1] = None
            cursor.updateRow(row)

In [31]:
# 7. Enter 'Station'
searchStns = "APLSTN|CLPSTN|CSFSTN|ANGSTN|CRKSTN|CIASTN"
for feature in in_features:
    with arcpy.da.UpdateCursor(feature,['DocName','Station','CP']) as cursor:
        for row in cursor:
            if row[0]:
                try:
                    reg = re.search(searchStns,row[0]).group()
                except AttributeError:
                    reg = re.search(searchStns,row[0])
                
                # Station name
                if reg == "APLSTN":
                    row[1] = 7
                    row[2] = "N-01"
                elif reg == "CLPSTN":
                    row[1] = 8
                    row[2] = "N-01"
                elif reg == "CSFSTN":
                    row[1] = 6
                    row[2] = "N-02"
                elif reg == "ANGSTN":
                    row[1] = 5
                    row[2] = "N-03"
                elif reg == "CRKSTN":
                    row[1] = 4
                    row[2] = "N-03"
                elif reg == "CIASTN":
                    row[1] = 3
                    row[2] = "N-04"
            else:
                row[1] = None
                row[2] = None
            cursor.updateRow(row)

In [32]:
# 8. Enter CP
for feature in in_features:
    with arcpy.da.UpdateCursor(feature,['DocName','CP']) as cursor:
        for row in cursor:
            if row[0]:
                try:
                    reg = re.search(searchStns,row[0]).group()
                except AttributeError:
                    reg = re.search(searchStns,row[0])
                
                # Station name
                if reg == "APLSTN":
                    row[1] = "N-01"
                elif reg == "CLPSTN":
                    row[1] = "N-01"
                elif reg == "CSFSTN":
                    row[1] = "N-02"
                elif reg == "ANGSTN":
                    row[1] = "N-03"
                elif reg == "CRKSTN":
                    row[1] = "N-03"
                elif reg == "CIASTN":
                    row[1] = "N-04"
            else:
                row[1] = None
            cursor.updateRow(row)

In [5]:
# 9. Apply symbology to Status
symbolLyrx = r'C:/Users/eiji.LAPTOP-KSD9P6CP/Dropbox/01-Railway/02-NSCR-Ex/10-ArcGIS/01-Reference/01-Layer File/03-Station_Structure/Station_Structure_FINAL.lyrx'

for feature in in_features:
    arcpy.ApplySymbologyFromLayer_management(feature, symbolLyrx, [["VALUE_FIELD", "Status", "Status"]], update_symbology="MAINTAIN")[0]

In [ ]:
# 10.Assign Domains
# 0. Domain Name
domainList = ['Station Structures_TYPE', 'Station Structures_SUBTYPE',\
              'Station Structures_STATUS', 'Station Structures_STRUCTURETYPE',\
              'Station Structures_STRUCTURELEVEL','Station_nscrex']

#domains = arcpy.da.ListDomains(workspace)
#listDomain = [d.name for d in domains]
#reg = re.compile(r'Station Structures*|Station*')
#listFinal = list(filter(reg.match, listDomain))
for feature in in_features:
    arcpy.AssignDomainToField_management(feature, 'Types', domainList[0])
    arcpy.AssignDomainToField_management(feature, 'SubType', domainList[1])
    arcpy.AssignDomainToField_management(feature, 'Status', domainList[2])
    arcpy.AssignDomainToField_management(feature, 'StructureType', domainList[3])
    arcpy.AssignDomainToField_management(feature, 'StructureLevel', domainList[4])
    arcpy.AssignDomainToField_management(feature, 'Station', domainList[5])

In [ ]:
# Assign Type based on Category field
### Domain Type (corresponding Category):
## 1. Structural Foundations (Structural Foundations)
## 2. Structural Columns (Structural Columns)
## 3. Structural Framing (Structural Framing)
## 4. Roofs (Roofs)
## 5. Floors (Floors)
## 6. Walls (Walls)
## 7. Columns (Columns)
## 8. Others

## note that 4 to 8 fall under Architectural disciplines of Structural revit file
for feature in in_features:
    with arcpy.da.UpdateCursor(feature, ['Category', 'Types']) as cursor:
        for row in cursor:
            if row[0] == 'Structural Foundations':
                row[1] = 1
            elif row[0] == 'Structural Columns':
                row[1] = 2
            elif row[0] == 'Structural Framing':
                row[1] = 3
            elif row[0] == 'Roofs':
                row[1] = 4
            elif row[0] == 'Floors':
                row[1] = 5
            elif row[0] == 'Walls':
                row[1] = 6
            elif row[0] == 'Columns':
                row[1] = 7
            else:
                row[1] = 8
            cursor.updateRow(row)

### SC Depot Buildings

In [5]:
from pathlib import Path
import os
import re

In [ ]:
homeDirectory = Path.home()
bimDirectory = r'Dropbox/01-Railway/02-NSCR-Ex/02-SC/01-Basedata/09-Building_Depot/01-BIM'
wd = os.path.join(homeDirectory, bimDirectory)

In [ ]:
# Output feature dataset from BIM
#!!!!! MAKE SURE THAT THE NAME DOES NOT OVERLAP. OTHERWISE, THE ORIGINAL WILL BE OVERWRITTEN.!!
#out_dataset_name = "SC_DEPOT_BIM"
out_dataset_name = "sc_depot_BIM_temp"

In [ ]:
# Extract folder names where each BIM model is stored
allFolders = []
inFolders = os.listdir(wd)
for folder in inFolders:
    reg = folder.endswith('.Rhistory')
    if reg:
        print("")
    else:
        allFolders.append(folder)

In [ ]:
## Use this code when required to choose only one or subset of folders of depot buildings
keywords = ['24_CWT', '26_DB']
temp = [f for f in allFolders if f in keywords]
allFolders = temp
allFolders

In [ ]:
# Compile all Structural discipline revit files
compile = []
compileDepNames = []
for folder in allFolders:
    files = os.listdir(os.path.join(wd, folder))
    n = 0
    for file in files:
        n = n + 1
        tt = file.endswith('MOD-ST-000001.rvt')
        if tt:
            print(file, end =' \n')
            compile.append(os.path.join(wd, folder, files[n-1]))
            compileDepNames.append(file)

In [ ]:
# Define directory of enterprise geodatabase
sde_name = "NSCR-Ex.sde"
out_gdb_path = os.path.join(homeDirectory,
                            "OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexsc",
                            sde_name)

spatial_reference = "PRS 1992 Philippines Zone III"

In [ ]:
# Run BIM File To Geodatabase
arcpy.BIMFileToGeodatabase_conversion(compile, out_gdb_path, out_dataset_name, spatial_reference, "")

In [ ]:
# Run Make Building layer
input_dataset = os.path.join(out_gdb_path, out_dataset_name)
arcpy.MakeBuildingLayer_management(input_dataset, out_dataset_name)

In [ ]:
# Drag and Drop all layers
inFeatures = ["sc_depot_BIM_temp\\Structural\\StructuralColumns",
              "sc_depot_BIM_temp\\Structural\\StructuralFoundation",
              "sc_depot_BIM_temp\\Architectural\\Floors",
              "sc_depot_BIM_temp\\Architectural\\Walls"]

In [ ]:
# Add Field Names
## As default, field name 'Type' exists so remove this first
for feature in inFeatures:
    arcpy.management.DeleteField(feature, 'Type')

addFields = ['Name', 'Type', 'SubType', 'Status', 'StartDate', 'TargetDate', 'CP', 'ID', 'Comment']

for feature in inFeatures:
    for field in addFields:
        if field in ('StartDate', 'TargetDate'):
            arcpy.AddField_management(feature, field, "DATE", "", "", "", field, "NULLABLE", "")
        elif field in ('CP', 'ID', 'Comment', 'Name'):
            arcpy.AddField_management(feature, field, "TEXT", "", "", "", field, "NULLABLE", "")
        else:
            arcpy.AddField_management(feature, field, "SHORT", "", "", "", field, "NULLABLE", "")

In [ ]:
# Set the initial status = 'To be Constructed'
for feature in inFeatures:
    with arcpy.da.UpdateCursor(feature, ['Status']) as cursor:
        for row in cursor:
            row[0] = 1
            cursor.updateRow(row)

In [ ]:
# Add CP = S-07
for feature in inFeatures:
    with arcpy.da.UpdateCursor(feature, ['CP']) as cursor:
        for row in cursor:
            row[0] = 'S-07'
            cursor.updateRow(row)

In [ ]:
# Assign Depot building names for each multipatch layer
separator = ';'
for feature in inFeatures:
    with arcpy.da.UpdateCursor(feature, ['DocName', 'Name']) as cursor:
        for row in cursor:
            if row[0] is None:
                print('no')
            else:
                tName = row[0]
                fName = tName[16:19]
                row[1] = fName
                cursor.updateRow(row)

In [ ]:
# Fix WP1 and WP2: correct ones are WPH1 and WPH2
for feature in inFeatures:
    with arcpy.da.UpdateCursor(feature, ['Name']) as cursor:
        for row in cursor:
            if row[0] == 'WP1':
                row[0] = 'WPH1'
            elif row[0] == 'WP2':
                row[0] = 'WPH2'
            cursor.updateRow(row)

In [ ]:
domains = arcpy.da.ListDomains(out_gdb_path)
#listDomain = [d.name for d in domains]
domainList = ['Station Structures_TYPE', 'Station Structures_STATUS', 'N2_Depot_Building_NAME']

for feature in inFeatures:
    arcpy.AssignDomainToField_management(feature, 'Type', domainList[0])
    arcpy.AssignDomainToField_management(feature, 'Status', domainList[1])
    arcpy.AssignDomainToField_management(feature, 'Name', domainList[2])

In [ ]:
# Assign Symbology
## 2. Apply symbolology
symbolLyrx = os.path.join(homeDirectory, "Dropbox/01-Railway/02-NSCR-Ex/10-ArcGIS/01-Reference/01-Layer File/03-Station_Structure/Station_Structure_FINAL.lyrx")

for feature in inFeatures:
    arcpy.ApplySymbologyFromLayer_management(feature, symbolLyrx, [["VALUE_FIELD", "Status", "Status"]], update_symbology="MAINTAIN")[0]

In [ ]:
# Assign Type based on Category field
### Domain Type (corresponding Category):
## 1. Structural Foundations (Structural Foundations)
## 2. Structural Columns (Structural Columns)
## 3. Structural Framing (Structural Framing)
## 4. Roofs (Roofs)
## 5. Floors (Floors)
## 6. Walls (Walls)
## 7. Columns (Columns)
## 8. Others

## note that 4 to 8 fall under Architectural disciplines of Structural revit file
for feature in inFeatures:
    with arcpy.da.UpdateCursor(feature, ['Category', 'Type']) as cursor:
        for row in cursor:
            if row[0] == 'Structural Foundations':
                row[1] = 1
            elif row[0] == 'Structural Columns':
                row[1] = 2
            elif row[0] == 'Structural Framing':
                row[1] = 3
            elif row[0] == 'Roofs':
                row[1] = 4
            elif row[0] == 'Floors':
                row[1] = 5
            elif row[0] == 'Walls':
                row[1] = 6
            elif row[0] == 'Columns':
                row[1] = 7
            else:
                row[1] = 8
            cursor.updateRow(row)